<a href="https://colab.research.google.com/github/userddssilva/ESTCMP246-mineracao-de-dados/blob/main/mineracao_de_dados_trabalho_1_extra%C3%A7%C3%A3o_de_aspectos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementação de regras para Extração de Aspectos em Sentenças

Universidade do Estado do Amazonas

Escola Superior de Tecnologia 

Disciplina de Mineração de dados


- Dayvson dos Santos Silva - ddss.snf19@uea.edu.br
- Bonifacio Leite De Oliveira Filho - bldof.eng16@uea.edu.br


## Instalandos as *libs* usadas no processo de extração

In [ ]:
# installing https://spacy.io/
!pip install --upgrade spacy 
!python -m spacy
!python -m spacy download pt_core_news_lg
!pip install senticnet
!python setup.py install
!python -m spacy download en_core_web_lg
# unicode
!pip install Unidecode

     |████████████████████████████████| 12.8MB 283kB/s 
     |████████████████████████████████| 9.1MB 33.0MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 1.1MB 30.0MB/s 
     |████████████████████████████████| 460kB 30.9MB/s 
     |████████████████████████████████| 122kB 38.1MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107107 sha256=664474f9cee93cd29a63ecec2c0fb1abdb2138e6ef94f600276b99eceb20d3d6
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: smart-open 5.0.0
    Uninstalling smart-open-5.0.0:
      Successfully uninstalled smart-open-5.0.0
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-

## Importando as *libs* para a realização do trabalho 

In [ ]:
import re
import pathlib
import glob

import pt_core_news_lg
import en_core_web_lg
import spacy

import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

from spacy import displacy
from senticnet.senticnet import SenticNet
from senticnet.babelsenticnet import BabelSenticNet

# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

# remove accents
from unidecode import unidecode

# Pretty print
from pprint import pprint

## Baixando o *dataset* do Google Drive

In [ ]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
# 
# Link file: https://drive.google.com/file/d/1xZ71zKdaflWO0mXNTzQEhXNdCcOEbZh3/view?usp=sharing
# Id file: 1xZ71zKdaflWO0mXNTzQEhXNdCcOEbZh3

file_id = '1xZ71zKdaflWO0mXNTzQEhXNdCcOEbZh3'
file_name = 'dataset.xml'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(filename=file_name)
print('Downloaded file "{}"'.format(file_name))

# Second alternative
# !gdown --id link_id_file

Downloaded file "dataset.xml"


## Carregando o pacote de núcleo do spacy em português

In [ ]:
nlp = pt_core_news_lg.load()

## Exemplo de *plot* de dependências

In [ ]:
sentence = "A câmera é boa."
doc = nlp(sentence)
displacy.render(doc, style='dep', jupyter=True)

In [ ]:
sentence = "o pobre manual"
doc = nlp(sentence)
displacy.render(doc, style='dep', jupyter=True)

## Importando o dataset do XML

Usando a [lib de xml](https://docs.python.org/3/library/xml.etree.elementtree.html) para trabalhar com o *dataset* que por sua vez está em xml.

In [ ]:
tree = ET.parse('dataset.xml')
dataset = tree.getroot()

Visualizando as tags que o arquivo contém

In [ ]:
dataset.tag

'reviews'

Visualizando os atributos que o arquivo possui

In [ ]:
len(dataset[0])

6

In [ ]:
len(dataset)

350

In [ ]:
print(dataset[0][0].tag)
print(dataset[0][1].tag)
print(dataset[0][2].tag)

sentence
opinion
opinion


## Extraíndo do dataset todas as sentenças não factuais e seus respectivos aspectos

**Passo 1:** Cria-se uma estrutura de dados contento o número da sentença e seus respectos aspectos em formato de lista, caso a sentença seja factual então a lista de aspectos é vazia

In [ ]:
# Get all sentences and yours respective aspects
new_dataset = {}
review_id = 0
k = 0
while review_id < len(dataset):
    review_line = 0
    while review_line < len(dataset[review_id]):
        review = dataset[review_id][review_line]
        # Check if review is a sentence
        if review.tag == 'sentence':
            sentence_text = review.text.replace('\"', '')
            new_dataset[k] = (sentence_text, []) # tuple -> (sentence, [aspects...])
            k += 1
        else:
            # get aspect from review
            aspect = re.search(r'aspect=\"([\w\s]+)\"', review.text).group(1)
            if aspect is not None:
                new_dataset[k-1][1].append(aspect) # add aspects of preview sentence 
        review_line += 1
    review_id += 1 
pprint(new_dataset)

{0: ('Ambiente agradável, bons pratos, comida saborosa , bom serviço, vale a '
     'pena conferir, bom tempero, recomendo',
     ['Ambiente', 'pratos', 'comida', 'serviço', 'tempero']),
 1: ('Lugar muito agradável, comida excelente é importante o bom atendimento '
     'dos garçons.',
     ['comida', 'Lugar', 'atendimento dos garçons']),
 2: ('Recomendo.', []),
 3: ('Pedimos um pirarucu ao molho de tucupi e não arrependemos', []),
 4: ('O Coqueiro verde serve uma deliciosa carne de sol, também contam com '
     'serviço de entrega o que é uma boa pedida para quem não quer sair de '
     'casa.. O serviço no restaurante é simples, a comida também, mas muito '
     'gostosa..',
     ['carne de sol', 'comida', 'serviço', 'serviço de entrega']),
 5: ('Ambiente confortável, ótimo atendimento com um garçon simpático, ele nos '
     'contou a história do empreendimento e sempre bem solícito.',
     ['Ambiente', 'atendimento', 'garçon', 'garçon']),
 6: ('Comida Boa por um bom preço.', ['Comid

In [ ]:
# Number sentences with factual sentences
len(new_dataset)

1049

**Passo 2:** Remove-se todas as sentenças factuais, ou seja, as que possuem a lista de aspectos vazios

In [ ]:
sentences = []
for number,(sentence, aspect_list) in new_dataset.items():
    if len(aspect_list) > 0:
        sentences.append(sentence)

In [ ]:
# Number sentences, before drop factuals sentences
len(sentences)

591

## Transformando as sentenças em conjunto

In [ ]:
def clear_aspects(aspects_list):
    """
    Remove all accents and turn aspect lower,
    params: list of aspects
    return: list of aspects without accensts and lower case
    """
    tmp = []
    for aspect in aspects_list:
        aspect = aspect.lower()
        aspect = unidecode(aspect)
        tmp.append(aspect)
    return tmp

In [ ]:
aspects = set()
for number,(sentence, aspect_list) in new_dataset.items():
    if len(aspect_list) > 0:
        cleared_aspects = clear_aspects(aspect_list)
        aspects.update(cleared_aspects)
print(f'Quantidades de aspectos = {len(aspects)}')

Quantidades de aspectos = 284


## Implementação das regras 

Carregando o core de português

In [ ]:
nlp = pt_core_news_lg.load()

### A primeira regra escolhida, foi a regra 7 amod(NN,OP) da tabela 3

In [ ]:
def rule_7(doc):
    aspects = []
    for i in doc:
        if i.dep_ == 'amod':
            if i.head.pos_ == 'NOUN':
                aspects.append(str(i.head))
    return aspects


sentence = 'Lugar especialmente agradável, atendimento excepcional e a comida deliciosa.'
doc = nlp(sentence)
rule_7(doc)

['Lugar', 'atendimento', 'comida']

### Exemplo de sentença para a regra 7

In [ ]:
displacy.render(doc, style='dep', jupyter=True)

### A próxima regra escolhida foi a 23 da tabela 4

In [ ]:
def rule_23(doc):
    aspects = []
    for index in range(len(doc)-1):
        if doc[index].pos_ == 'NOUN' and doc[index+1].pos_ == 'ADJ':
            aspects.append(str(doc[index]))
    return aspects

sentence = 'Comida simples e barata, para quem esta de passagem pelo centro de João Pessoa, eu recomendo.'
doc = nlp(sentence)
rule_23(doc)

['Comida']

### Exemplo da sentença 23

In [ ]:
displacy.render(doc, style='dep', jupyter=True)

### Seguido pela regra 60 tabela 5

In [ ]:
def rule_60(doc):
    aspects = []
    for index in range(len(doc)-3):
        if (doc[index].pos_ == 'DET') and (doc[index+1].pos_ == 'NOUN') and (doc[index+2].pos_ != 'NOUN' )and (doc[index+3].pos_ == 'ADJ'):
            aspects.append(str(doc[index+1]))
    return aspects

nlp = en_core_web_lg.load()
# their customer service is very poor.
sentence = 'this camera is closest to perfect'
doc = nlp(sentence)
print(rule_60(doc))
npl = pt_core_news_lg.load()

['camera']


### Exemplo de sentença usada na regra 60

In [ ]:
displacy.render(doc, style='dep', jupyter=True)

## Aplicando as regras de extração de sentenças



In [ ]:
def apply_rule(sentences, rule):
    """
    This function will be re applay a rule on sentence list.
    params: sentences list and rule function
    return: set of aspects
    """
    predicted_aspects = set()
    for sentence in sentences:
        pred_aspects = rule(nlp(sentence))
        pred_aspects = clear_aspects(pred_aspects) # clear aspects
        predicted_aspects.update(pred_aspects)
    return predicted_aspects

In [ ]:
# Applaying rules 7, 23 and 60 on sentences list
rule_7_aspects_predicts = apply_rule(sentences, rule_7)     # rule 7
rule_23_aspects_predicts = apply_rule(sentences, rule_23)   # rule 23
rule_60_aspects_predicts = apply_rule(sentences, rule_60)   # rule 60

In [ ]:
print(f'Quantidade de aspectos extraídos pela regra 7  = {len(rule_7_aspects_predicts)}')
print(f'Quantidade de aspectos extraídos pela regra 23 = {len(rule_23_aspects_predicts)}')
print(f'Quantidade de aspectos extraídos pela regra 60 = {len(rule_60_aspects_predicts)}')

Quantidade de aspectos extraídos pela regra 7  = 116
Quantidade de aspectos extraídos pela regra 23 = 15
Quantidade de aspectos extraídos pela regra 60 = 3


## Avaliando a extração de aspectos pelas regras

Unindo os conjuntos de aspectos preditos em cada uma das regras implementadas

In [ ]:
pred_aspects = set.union(rule_7_aspects_predicts, rule_23_aspects_predicts, rule_60_aspects_predicts)

Quantidade de aspectos preditos nas três regras implementadas

In [ ]:
len(pred_aspects)

128

## Métricas

In [ ]:
# Precision
def precision(aspects_real, aspects_pred):
    number_correct_aspects = 0
    number_pred_aspects = len(aspects_pred)

    for pred_aspect in aspects_pred:
        if pred_aspect in aspects_real:
            number_correct_aspects += 1

    return number_correct_aspects / number_pred_aspects

In [ ]:
# Recall
def recall(aspects_real, aspects_pred):
    number_correct_aspects = 0
    number_aspects = len(aspects_real)

    for pred_aspect in aspects_pred:
        if pred_aspect in aspects_real:
            number_correct_aspects += 1

    return number_correct_aspects / number_aspects

In [ ]:
# F1-Score
def f1_score(aspects_real, aspects_pred):
    p = precision(aspects_real, aspects_pred)
    r = recall(aspects_real, aspects_pred)
    return (2 * p * r) / (p + r)

### Métricas com os aspectos extraídos pelas três regras

In [ ]:
print('Precision: ', precision(aspects, pred_aspects))
print('Recall: ', recall(aspects, pred_aspects))
print('F1-Score: ', f1_score(aspects, pred_aspects))

Precision:  0.1484375
Recall:  0.06690140845070422
F1-Score:  0.09223300970873786


### Métricas para a regra 7

In [ ]:
print('Precision: ', precision(aspects, rule_7_aspects_predicts))
print('Recall: ', recall(aspects, rule_7_aspects_predicts))
print('F1-Score: ', f1_score(aspects, rule_7_aspects_predicts))

Precision:  0.13793103448275862
Recall:  0.056338028169014086
F1-Score:  0.07999999999999999


### Métricas para a regra 23

In [ ]:
print('Precision: ', precision(aspects, rule_23_aspects_predicts))
print('Recall: ', recall(aspects, rule_23_aspects_predicts))
print('F1-Score: ', f1_score(aspects, rule_23_aspects_predicts))

Precision:  0.26666666666666666
Recall:  0.014084507042253521
F1-Score:  0.026755852842809368


### Métricas para a regra 60

In [ ]:
print('Precision: ', precision(aspects, rule_60_aspects_predicts))
print('Recall: ', recall(aspects, rule_60_aspects_predicts))
print('F1-Score: ', f1_score(aspects, rule_60_aspects_predicts))

Precision:  0.3333333333333333
Recall:  0.0035211267605633804
F1-Score:  0.006968641114982579
